In [1]:
import pandas as pd
import numpy as np
from numpy import asarray

from PIL import Image


from tqdm import tqdm
from tqdm import tqdm_notebook

from numpy import expand_dims
from numpy import savez_compressed

import ast


In [2]:
#Load the Dataframes with embeddings
photo_df_a = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_a.csv')
photo_df_b = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_b.csv')
photo_df_c = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_c.csv')
photo_df_d = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_d.csv')
photo_df_e = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_e.csv')
photo_df_f = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_f.csv')
photo_df_g = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_g.csv')
photo_df_h = pd.read_csv('photo_info_df/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_h.csv')

In [8]:
photo_df_h.loc[7428]


name                                                   Noel Redding
dob                                                      1945-12-25
gender                                                            1
photo_taken                                                    1967
age_when_taken                                                   22
file_path         image_data/wiki_data/45/226445_1945-12-25_1967...
face_location     [172.48375401 129.6128155  248.65144331 205.78...
face_score                                                   1.2101
embeddings_fn     [nan nan nan nan nan nan nan nan nan nan nan n...
Name: 7428, dtype: object

In [3]:
#Upon further inspection row 7428 in Dataframe h is the only one that breaks in my conversion function,
#this is because nans were returned by the model, I looked into this further and found that the picture
#is just a white square, I remove this row here
photo_df_h.drop(photo_df_h.index[7428], inplace = True)


#Concat the dataframes
photo_df = pd.concat([photo_df_a, photo_df_b, photo_df_c, photo_df_d, photo_df_e, photo_df_f, photo_df_g, photo_df_h]).reset_index(drop=True)

In [4]:
photo_df.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,embeddings_fn
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm124825600_...,[1072.926 161.838 1214.784 303.696],1.459693,[-0.74029577 0.14542007 -0.5782049 -0.007681...
1,Fred Astaire,1899-05-10,1.0,1970,71,image_data/imdb_data/01/nm0000001_rm3343756032...,[477.184 100.352 622.592 245.76 ],2.543198,[-0.1463713 -0.5106522 -0.48219085 -0.486318...
2,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm577153792_...,[114.96964309 114.96964309 451.68657236 451.68...,3.455579,[ 0.35580358 0.14148678 0.11161471 -0.294302...
3,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117,[ 0.1044133 -0.80462444 -0.86017245 -0.593098...
4,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766,[-1.3740315 -0.807325 -0.21413553 0.019938...


In [5]:
#This function converts the saved string in the 'embeddings_fn' column to a numpy array

def convert_csv_to_embeddings(embedding_string):
    
    #I replace the '\n' and spaces in descending sequential order
    embedding_string = embedding_string.replace('\n', '').replace('     ', ' ').replace('    ', ' ').replace('   ', ' ').replace('  ', ' ').replace('[ ', '[').replace(' ]', ']').replace(' ', ', ')
    
    #This returns the string as an array in the proper type
    return asarray(ast.literal_eval(embedding_string)).astype('float32')





In [6]:
tqdm.pandas()
photo_df.embeddings_fn = photo_df.embeddings_fn.progress_apply(lambda x: convert_csv_to_embeddings(x))



100%|██████████| 372916/372916 [02:25<00:00, 2562.10it/s]


In [7]:
photo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372916 entries, 0 to 372915
Data columns (total 9 columns):
name              372916 non-null object
dob               372916 non-null object
gender            372916 non-null float64
photo_taken       372916 non-null int64
age_when_taken    372916 non-null int64
file_path         372916 non-null object
face_location     372916 non-null object
face_score        372916 non-null float64
embeddings_fn     372916 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 25.6+ MB


In [8]:
photo_meta = pd.read_csv('Photo_Dataframes/All_Photo_Data.csv')

In [9]:
photo_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264534 entries, 0 to 264533
Data columns (total 8 columns):
name              264534 non-null object
dob               264534 non-null object
gender            264534 non-null float64
photo_taken       264534 non-null int64
age_when_taken    264534 non-null int64
file_path         264534 non-null object
face_location     264534 non-null object
face_score        264534 non-null float64
dtypes: float64(2), int64(2), object(4)
memory usage: 16.1+ MB


In [10]:
photo_reduced = pd.merge(photo_meta, photo_df, on = "file_path")
photo_reduced = photo_reduced[['name_x', 'dob_x', 'gender_x', 'photo_taken_x', 'age_when_taken_x', 'file_path', 'face_location_x', 'face_score_x', 'embeddings_fn']]
photo_reduced.rename(columns = {'name_x': 'name', 'dob_x': 'dob', 'gender_x': 'gender', 'photo_taken_x': 'photo_taken', 'age_when_taken_x': 'age_when_taken', 'face_location_x': 'face_location', 'face_score_x': 'face_score'}, inplace = True)
photo_reduced.head()

,name,dob,gender,photo_taken,age_when_taken,file_path,face_location,face_score,embeddings_fn
0,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm946909184_...,[622.88550564 424.21750384 844.33900767 645.67...,1.872117,"[0.1044133, -0.80462444, -0.86017245, -0.59309..."
1,Fred Astaire,1899-05-10,1.0,1968,69,image_data/imdb_data/01/nm0000001_rm980463616_...,[1013.85900236 233.88204221 1201.5861278 42...,1.158766,"[-1.3740315, -0.807325, -0.21413553, 0.0199386..."
2,Lauren Bacall,1924-09-16,0.0,1991,67,image_data/imdb_data/02/nm0000002_rm1075631616...,[ 686.67682059 458.46721373 1198.10043604 96...,3.490563,"[-2.134354, 0.60471165, 0.35341397, -1.1716923..."
3,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1346607872...,[ 78.23569407 52.33512938 194.25423516 168.35...,3.479189,"[-0.4142363, 0.04628806, -0.04605452, -1.31644..."
4,Lauren Bacall,1924-09-16,0.0,2004,80,image_data/imdb_data/02/nm0000002_rm1363385088...,[ 64.30867104 64.30867104 207.26768087 207.26...,3.870171,"[-0.18499404, 0.7116227, 0.44020605, -0.900509..."


In [11]:
photo_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264533 entries, 0 to 264532
Data columns (total 9 columns):
name              264533 non-null object
dob               264533 non-null object
gender            264533 non-null float64
photo_taken       264533 non-null int64
age_when_taken    264533 non-null int64
file_path         264533 non-null object
face_location     264533 non-null object
face_score        264533 non-null float64
embeddings_fn     264533 non-null object
dtypes: float64(2), int64(2), object(5)
memory usage: 20.2+ MB


In [12]:
photo_df_a = photo_reduced[:50000]
photo_df_b = photo_reduced[50000:100000]
photo_df_c = photo_reduced[100000:150000]
photo_df_d = photo_reduced[150000:200000]
photo_df_e = photo_reduced[200000:250000]
photo_df_f = photo_reduced[250000:]

In [13]:
#Save the Dataframes with embeddings
photo_df_a.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_a.csv', index = False)
photo_df_b.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_b.csv', index = False)
photo_df_c.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_c.csv', index = False)
photo_df_d.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_d.csv', index = False)
photo_df_e.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_e.csv', index = False)
photo_df_f.to_csv('Photo_Dataframes/photo_dfs_with_embeddings_fn/photo_dataframe_fn_embeddings_f.csv', index = False)